In [1]:
import pexpect
import time
import sys
import re
import datetime
import os

In [2]:
child = pexpect.spawn('ssh -L 7898:localhost:8007 -J root@ip.numbers root@127.0.0.1 -p 8007', timeout=30, encoding='utf-8', logfile=sys.stdout)

child.maxread = 300000 

child.expect("root@ip.number's password: ")
child.sendline('') #Enters Password
child.expect("root@127.0.0.1's password: ")
child.sendline('') #Enters Password

time.sleep(10)

# Shell command to take after the dates in modification dates
# child.sendline('find /var/opt/common5/mdt/ -type f -newermt 2023-04-16 -exec ls -l {} \;')

# Shell command to take only the folder
# child.sendline('find /var/opt/common5/mdt/20230205 -type f -exec ls -l {} \;')

# Shell command to take certain range dates in modification dates
child.sendline('find /var/opt/common5/mdt -type f -newermt 2023-08-01 ! -newermt 2023-09-01 -exec ls -l {} \;')

try:
    child.expect(pexpect.EOF, timeout=100)

except pexpect.TIMEOUT:
    child.close()
    raw = str(child.before)

root@150.236.203.198's password: PMT@2022

root@127.0.0.1's password: PMT@2022
find /var/opt/common5/mdt -type f -newermt 2023-08-01 ! -newermt 2023-09-01 -exec ls -l {} \;

Last login: Wed Sep  6 14:22:37 2023 from 192.168.21.6
! -newermt 2023-09-01 -exec ls -l {} \;-2 ~]# find /var/opt/common5/mdt -type f -newermt 2023-08-01  
-rw-r--r--. 1 root root 1261674 Aug  1 00:33 /var/opt/common5/mdt/20230731/19/result_ue_cap.zip
-rw-r--r--. 1 root root 42171637 Aug  1 00:35 /var/opt/common5/mdt/20230731/19/ue_traffic.zip
-rw-r--r--. 1 root root 87290326 Aug  1 00:37 /var/opt/common5/mdt/20230731/19/ue_meas.zip
-rw-r--r--. 1 root root 56654123 Aug  1 00:41 /var/opt/common5/mdt/20230731/19/mdt_result.zip
-rw-r--r--. 1 root root 5624243 Aug  1 00:42 /var/opt/common5/mdt/20230731/19/ta_histogram.zip
-rw-r--r--. 1 root root 911942 Aug  1 00:55 /var/opt/common5/mdt/20230731/19/ho_counter_20230731_19.zip
-rw-r--r--. 1 root root 1313921 Aug  1 01:33 /var/opt/common5/mdt/20230731/20/result_ue_cap.zip

In [ ]:
test1 = re.split(r'\s+|\n', raw)

In [ ]:
# Create list comprehension to filter list of zip files only

# result = [file for file in test1 if file.endswith('.zip')]

# Create list comprehension to filter only mdt_result.zip only
# Option are ue_traffic or mdt_result

pattern = r'.*/(?!filter_)[^/]+/ue_traffic\.zip$'
result = [path for path in test1 if re.match(pattern, path)]

In [ ]:
# Create list comprehension to generate weekend dates in June

weekend_dates = [date.strftime("%Y%m%d") for date in (datetime.date(2023, 8, day) for day in range(1, 32)) 
                 if date.weekday() >= 5]

# Create list comprehension to generate continous dates in June (from 25 June until 30 June)
# continous_dates = re.compile(r'/var/opt/common5/mdt/202306(2[5-9]|30)/\d{2}/ue_traffic\.zip')

In [ ]:
# Create list comprehension to filter June Month only
result = [path for path in result if '202308' in path]

# Create list comprehension to filter Out Some Date
result = [path for path in result if '20230817' not in path]   

# Create list comprehension to filter out continous dates
# result = [path for path in result if not continous_dates.search(path)]

# Create list comprehension to filter Out Weekend Date
result = [path for path in result if path.split('/')[5] not in weekend_dates]

# Create list comprehension to filter Busy Hour only
result = [path for path in result if '/18/' in path or '/19/' in path or '/20/' in path or '/21/' in path or '/22/' in path]

# Create list comprehension to filter Busy Hour only
# result = [path for path in result if '/11/' in path or '/12/' in path or '/13/' in path]

# result = [path for path in result if '/07/' in path or '/08/' in path or '/09/']

result.sort()

In [ ]:
with open(r'/home/nivag/location_to_dl_8008-throughput.txt', 'w') as fp:
    fp.write('\n'.join(result))